In [302]:
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm

In [36]:
import os
import glob

In [280]:
DATA_DIR = "transparent_california_data"
data_paths = glob.glob(os.path.join(DATA_DIR, "university-of-california-*.csv"))
PHYSICIAN_SALARY_DATA_PATH = os.path.join(DATA_DIR, "physcian_salaries.csv")
PHYSICIAN_JOB_TITLES_PATH = os.path.join(DATA_DIR, "physician_job_titles.txt")

In [285]:
def load_physician_job_titles():
    """"""
    with open(PHYSICIAN_JOB_TITLES_PATH) as file:
        return sorted(set([title.lower() for title in file.read().split("\n")]))
    
physician_job_titles = load_physician_job_titles()

In [286]:
def clean_name(name: str) -> str:
    """"""
    name = name.lower()
    name = name.strip(", md").strip(", dr.")
    if "," in name:
        splits = name.split(",")
        if len(splits) > 2:
            return name
        assert len(splits) == 2, f"Should only have one , in name if any. {name}"
        name = splits[1].strip() + " " + splits[0].strip()
    return name

In [287]:
import gender_guesser.detector as gender
gd = gender.Detector(case_sensitive=False)

def guess_gender(name: str) -> str:
    """"""
    first_name = name.split(" ")[0]
    return gd.get_gender(first_name)

In [324]:
def load_salary_data(data_path: str) -> pd.DataFrame:
    """"""
    salary_data = pd.read_csv(data_path, low_memory=False)
    salary_data.columns = [col.lower().strip().replace(" ", "_") for col in salary_data.columns]
    salary_data["pension_debt"] = 0
    salary_data.drop(columns=["pension_debt", "notes", "status", "agency"], inplace=True)
    salary_data.query('base_pay > 50_000', inplace=True)
    salary_data["employee_name"] = salary_data["employee_name"].apply(clean_name)
    salary_data.query("employee_name != 'not provided'", inplace=True)
    salary_data["gender"] = salary_data["employee_name"].apply(guess_gender)
    salary_data["job_title"] = salary_data["job_title"].str.lower()
    
    return salary_data

In [328]:
def load_physician_salary_data(overwrite: bool = False) -> pd.DataFrame:
    """"""
    if os.path.exists(PHYSICIAN_SALARY_DATA_PATH) and not overwrite:
        physician_salary_data = pd.read_csv(PHYSICIAN_SALARY_DATA_PATH)
    else:
        all_salary_data = pd.concat([load_salary_data(data_path) for data_path in data_paths])
        physician_salary_data = all_salary_data.loc[all_salary_data["job_title"].isin(physician_job_titles)]
        physician_salary_data.to_csv(PHYSICIAN_SALARY_DATA_PATH, index=False)
    
    return physician_salary_data

In [331]:
physician_salary_data = load_physician_salary_data()

In [333]:
is_resident = physician_salary_data["job_title"].str.contains("resid phys")
is_instr = physician_salary_data["job_title"].str.contains("instr")
is_attending = ~(is_resident | is_instr)

resident_data = physician_salary_data.loc[is_resident].query("year == 2019")
attending_data = physician_salary_data.loc[is_attending].query("year > 2018")

In [334]:
resident_names = resident_data["employee_name"]
resident_and_attending_names = resident_names.loc[resident_names.isin(attending_data["employee_name"])]

repeat_data = physician_salary_data.loc[physician_salary_data["employee_name"].isin(resident_and_attending_names)].sort_values(["employee_name", "year"])
duplicated_names = repeat_data["employee_name"].loc[repeat_data.duplicated(["employee_name", "year"])]
repeat_data.loc[~repeat_data["employee_name"].isin(duplicated_names)]

,employee_name,job_title,base_pay,overtime_pay,other_pay,benefits,total_pay,total_pay_&_benefits,year,gender
87420,andall lee,resid phys ii-viii/non rep,63778.0,0.0,5000.0,0.0,68778.0,68778.0,2019,unknown
94838,andall lee,prof of clin-hcomp,137143.0,0.0,94127.0,20847.0,231270.0,252117.0,2020,unknown
85092,arisa hernandez-morgan,resid phys/subspec 4-8/non rep,78038.0,0.0,63296.0,6520.0,141334.0,147854.0,2019,female
90776,arisa hernandez-morgan,hs asst clin prof-hcomp,154950.0,0.0,278065.0,38226.0,433015.0,471241.0,2020,female
86589,atthew russell,resid phys/subspec 4-8/non rep,66940.0,0.0,2747.0,8703.0,69687.0,78390.0,2019,unknown
91891,atthew russell,hs assoc clin prof-hcomp,168166.0,0.0,203787.0,17195.0,371953.0,389148.0,2020,unknown
85653,bi mo,resid phys/subspec 4-8/non rep,74369.0,0.0,32133.0,7407.0,106502.0,113909.0,2019,andy
90921,bi mo,hs asst clin prof-hcomp,158978.0,0.0,254576.0,43052.0,413554.0,456606.0,2020,andy
87769,eric chang,resid phys ii-viii/non rep,62084.0,0.0,5250.0,0.0,67334.0,67334.0,2019,male
101676,eric chang,hs clin prof-hcomp,56391.0,0.0,0.0,5884.0,56391.0,62275.0,2020,male
